In [3]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
import yfinance as yf
from pandas.tseries.offsets import BusinessDay
import concurrent.futures
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import os

env_path = '../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

In [4]:
project = hopsworks.login(project='AMomozZz', api_key_value=HOPSWORKS_API_KEY)

2024-12-30 18:04:14,403 INFO: Initializing external client
2024-12-30 18:04:14,404 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-30 18:04:15,568 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163416


## Process data

---

In [4]:
today = datetime.date.today()

startDate="2000-08-30"
endDate = today.isoformat()
base = "USD"

In [4]:
# Fetch data
url = f"https://api.frankfurter.dev/v1/{startDate}..{endDate}?base={base}"

response = requests.get(url)
data = response.json()

# Convert JSON to DataFrame
df = pd.DataFrame.from_dict(data["rates"], orient="index")

# Add a 'Date' column as the first column
df.index.name = "Date"
df.reset_index(inplace=True)

# Save to CSV
df.to_csv("../data/exchangeRates.csv", index=False)

print("CSV file saved as '../data/exchangeRates.csv'")

CSV file saved as '../data/exchangeRates.csv'


In [5]:
exchangeRates_csv_file="../data/exchangeRates.csv"
util.check_file_path(exchangeRates_csv_file)

File successfully found at the path: ../data/exchangeRates.csv


In [6]:
# Use the correct ticker symbol for gold, for example, 'GC=F' (Gold Futures)
gold_ticker = 'GC=F'
# Fetch historical data
gold_data = yf.download(gold_ticker, start=startDate, end=endDate)

# Save to CSV
gold_csv_path = '../data/goldPrices.csv'
gold_data.to_csv(gold_csv_path)

[*********************100%***********************]  1 of 1 completed


In [7]:
goldPrices_csv_file="../data/goldPrices.csv"
util.check_file_path(goldPrices_csv_file)

File successfully found at the path: ../data/goldPrices.csv


In [8]:
df_exchange = pd.read_csv(exchangeRates_csv_file)

# remove cloum with NaN
non_nan_columns = df_exchange.dropna(axis=1)

non_nan_columns.columns = non_nan_columns.columns.str.lower().str.replace(' ', '_')

non_nan_columns.to_csv("../data/exchangeRates_filtered.csv", index=False)

In [9]:
df_gold = pd.read_csv(goldPrices_csv_file)

df_gold = df_gold.iloc[2:].reset_index(drop=True)

new_columns = ["Date"] + [col + " Price" for col in df_gold.columns[1:]]
df_gold.columns = new_columns

df_gold.to_csv("../data/goldPrices_modified.csv", index=False)

df_filtered = df_gold[["Date", "Close Price"]]

df_filtered.columns = df_filtered.columns.str.lower().str.replace(' ', '_')

df_filtered.to_csv("../data/goldPrices_date_close.csv", index=False)

## Get filtered data

---

In [5]:
df_gold = pd.read_csv("../data/goldPrices_date_close.csv")
df_exchange = pd.read_csv("../data/exchangeRates_filtered.csv")

In [6]:
df_gold.info()
df_exchange.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6104 entries, 0 to 6103
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         6104 non-null   object 
 1   close_price  6104 non-null   float64
dtypes: float64(1), object(1)
memory usage: 95.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6228 entries, 0 to 6227
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    6228 non-null   object 
 1   aud     6228 non-null   float64
 2   bgn     6228 non-null   float64
 3   cad     6228 non-null   float64
 4   chf     6228 non-null   float64
 5   czk     6228 non-null   float64
 6   dkk     6228 non-null   float64
 7   eur     6228 non-null   float64
 8   gbp     6228 non-null   float64
 9   hkd     6228 non-null   float64
 10  huf     6228 non-null   float64
 11  jpy     6228 non-null   float64
 12  krw     6228 non-null   float64
 13  nok   

## Add back 10 days exchange rate data

---

In [6]:
df_gold['date'] = pd.to_datetime(df_gold['date'])
df_exchange['date'] = pd.to_datetime(df_exchange['date'])

In [7]:
df_only_exchange = df_exchange.copy()

In [8]:
result = df_only_exchange.copy()
for i in range(1, 11):
    df_temp = df_only_exchange.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']) + BusinessDay(i)
    df_temp = df_temp.rename(columns=lambda x: f"{x.lower()}_{i}" if x != "date" else x)
    result = pd.merge(result, df_temp, how='left', on="date")

result.fillna(0, inplace=True)

In [9]:
pd.set_option('display.max_rows', 20)  
pd.set_option('display.max_columns', 20)  
result.sort_values(by='date', ascending=True).tail(10)

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,hkd_10,huf_10,jpy_10,krw_10,nok_10,nzd_10,pln_10,sek_10,sgd_10,zar_10
6218,2024-12-12,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,...,7.7828,391.61,151.67,1396.59,11.0643,1.6991,4.0884,10.9443,1.3437,18.1522
6219,2024-12-13,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,7.7837,389.84,150.20,1397.57,11.0590,1.6922,4.0674,10.9051,1.3406,18.0626
6220,2024-12-16,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,7.7808,394.93,150.13,1404.33,11.0964,1.6981,4.0835,10.9708,1.3456,18.1820
6221,2024-12-17,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,7.7833,394.45,149.64,1403.06,11.0626,1.7002,4.0880,11.0117,1.3451,18.1035
6222,2024-12-18,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,7.7840,394.96,151.15,1417.54,11.0756,1.7144,4.0858,11.0065,1.3472,18.1738
6223,2024-12-19,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,7.7824,390.70,150.40,1418.04,11.0626,1.7064,4.0555,10.9198,1.3409,18.0671
6224,2024-12-20,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,7.7794,391.60,150.51,1419.74,11.0996,1.7118,4.0327,10.8903,1.3396,18.0584
6225,2024-12-23,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,7.7748,389.15,150.83,1432.14,11.1123,1.7061,4.0304,10.9160,1.3396,17.8194
6226,2024-12-24,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,7.7760,390.57,151.74,1431.85,11.1442,1.7178,4.0541,10.9561,1.3411,17.8782
6227,2024-12-27,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866


In [11]:
a = result.sort_values(by='date', ascending=True).tail(1)

In [12]:
a["aud"], a["aud_1"], a["aud_2"], a["aud_3"], a["aud_4"], a["aud_5"], a["aud_6"], a["aud_7"], a["aud_8"], a["aud_9"], a["aud_10"]

(6227    1.6078
 Name: aud, dtype: float64,
 6227    0.0
 Name: aud_1, dtype: float64,
 6227    0.0
 Name: aud_2, dtype: float64,
 6227    1.6047
 Name: aud_3, dtype: float64,
 6227    1.6037
 Name: aud_4, dtype: float64,
 6227    1.6048
 Name: aud_5, dtype: float64,
 6227    1.5999
 Name: aud_6, dtype: float64,
 6227    1.5817
 Name: aud_7, dtype: float64,
 6227    1.5764
 Name: aud_8, dtype: float64,
 6227    1.5727
 Name: aud_9, dtype: float64,
 6227    1.5676
 Name: aud_10, dtype: float64)

## Add back 10 days gold price data

In [10]:
for i in range(1, 11):
    df_temp = df_gold.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']) + BusinessDay(i)
    df_exchange = pd.merge(df_exchange, df_temp[['date','close_price']], how='left', on="date")
    df_exchange = df_exchange.rename(columns={"close_price": f"close_price_{i}"})

df_exchange.fillna(0, inplace=True)

In [11]:
df_exchange.sort_values(by='date', ascending=True).tail(10)

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
6218,2024-12-12,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,...,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902,2657.000000,0.000000
6219,2024-12-13,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902,2657.000000
6220,2024-12-16,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902
6221,2024-12-17,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951
6222,2024-12-18,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049
6223,2024-12-19,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098
6224,2024-12-20,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098
6225,2024-12-23,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902
6226,2024-12-24,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098
6227,2024-12-27,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000


In [15]:
df_exchange.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6228 entries, 0 to 6227
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            6228 non-null   datetime64[ns]
 1   aud             6228 non-null   float64       
 2   bgn             6228 non-null   float64       
 3   cad             6228 non-null   float64       
 4   chf             6228 non-null   float64       
 5   czk             6228 non-null   float64       
 6   dkk             6228 non-null   float64       
 7   eur             6228 non-null   float64       
 8   gbp             6228 non-null   float64       
 9   hkd             6228 non-null   float64       
 10  huf             6228 non-null   float64       
 11  jpy             6228 non-null   float64       
 12  krw             6228 non-null   float64       
 13  nok             6228 non-null   float64       
 14  nzd             6228 non-null   float64       
 15  pln 

In [12]:
import great_expectations as ge

ge.core.ExpectationConfiguration

def expect_greater_than_zero(suite, col):
    suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":0,
                "strict":False
            }
        )
    )

gold_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="gold_expectation_suite"
)

only_exchange_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="only_exchange_expectation_suite"
)

exchange_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="exchange_expectation_suite"
)


expect_greater_than_zero(gold_expectation_suite, "close_price")

lst = [item for item in result.columns if item != 'date']
for i in lst:
    expect_greater_than_zero(only_exchange_expectation_suite, i)

lst = [item for item in df_exchange.columns if item != 'date']
for i in lst:
    expect_greater_than_zero(exchange_expectation_suite, i)

## Merge to feature store

In [13]:
fs = project.get_feature_store() 

In [14]:
gold_fg = fs.get_or_create_feature_group(
    name='gold_feature_group',
    description='gold feature group',
    version=1,
    primary_key=['date'],
    event_time="date",
    expectation_suite=gold_expectation_suite
)

In [15]:
exchange_fg = fs.get_or_create_feature_group(
    name='exchange_feature_group',
    description='exchange feature group',
    version=1,
    primary_key=['date'],
    event_time="date",
    expectation_suite=exchange_expectation_suite
) 

In [16]:
only_exchange_fg = fs.get_or_create_feature_group(
    name='only_exchange_feature_group',
    description='only exchange feature group(no gold prices)',
    version=1,
    primary_key=['date'],
    event_time="date",
    expectation_suite=only_exchange_expectation_suite
) 

In [26]:
def insert_data(feature_group, data):
    return feature_group.insert(data, write_options={"wait_for_job": True})

with concurrent.futures.ThreadPoolExecutor() as executor:
    future1 = executor.submit(insert_data, gold_fg, df_gold)
    future2 = executor.submit(insert_data, exchange_fg, df_exchange)
    future3 = executor.submit(insert_data, only_exchange_fg, result)

    concurrent.futures.wait([future1, future2, future3])


2024-12-29 01:45:03,670 INFO: 	1 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393256
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393338
2024-12-29 01:45:06,095 INFO: 	28 expectation(s) included in expectation_suite.
Validation failed.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393339
2024-12-29 01:45:07,764 INFO: 	198 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393338


Uploading Dataframe: 100.00% |██████████| Rows 6104/6104 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: gold_feature_group_1_offline_fg_materialization


Uploading Dataframe: 0.00% |          | Rows 0/6228 | Elapsed Time: 00:00 | Remaining Time: ?

Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393339


Uploading Dataframe: 100.00% |██████████| Rows 6228/6228 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: exchange_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1163416/jobs/named/gold_feature_group_1_offline_fg_materialization/executions


Uploading Dataframe: 100.00% |██████████| Rows 6228/6228 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: only_exchange_feature_group_1_offline_fg_materialization
2024-12-29 01:45:19,389 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1163416/jobs/named/exchange_feature_group_1_offline_fg_materialization/executions
2024-12-29 01:45:22,559 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-29 01:45:22,722 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1163416/jobs/named/only_exchange_feature_group_1_offline_fg_materialization/executions
2024-12-29 01:45:25,874 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-29 01:45:27,985 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-29 01:45

In [17]:
display(gold_fg.read().sort_values(by=['date'], ascending=True).tail(10))
display(exchange_fg.read().sort_values(by=['date'], ascending=True).tail(10))
display(only_exchange_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.26s) 


,date,close_price
2935,2024-12-13 00:00:00+00:00,2656.000000
5357,2024-12-16 00:00:00+00:00,2651.399902
3292,2024-12-17 00:00:00+00:00,2644.399902
2607,2024-12-18 00:00:00+00:00,2636.500000
5078,2024-12-19 00:00:00+00:00,2592.199951
5611,2024-12-20 00:00:00+00:00,2628.699951
1922,2024-12-23 00:00:00+00:00,2612.300049
1809,2024-12-24 00:00:00+00:00,2620.000000
6102,2024-12-26 00:00:00+00:00,2638.800049
6103,2024-12-27 00:00:00+00:00,2636.500000


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.04s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
5183,2024-12-19 00:00:00+00:00,1.599900,1.881500,1.436300,0.896490,24.172000,7.177800,0.962000,0.793120,7.771500,...,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098
5730,2024-12-20 00:00:00+00:00,1.604800,1.882400,1.438500,0.894800,24.178000,7.181300,0.962460,0.798510,7.772400,...,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098
1975,2024-12-23 00:00:00+00:00,1.603700,1.881800,1.441200,0.898300,24.213000,7.176900,0.962190,0.798570,7.770300,...,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902
1862,2024-12-24 00:00:00+00:00,1.604700,1.881500,1.441800,0.900240,24.180000,7.177300,0.962000,0.796580,7.767500,...,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098
2423,2024-12-27 00:00:00+00:00,1.607800,1.874300,1.437700,0.900430,24.150000,7.149200,0.958310,0.796340,7.761400,...,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000
6230,2024-12-30 00:00:00+00:00,1.541141,1.838320,1.379623,0.897305,24.417318,6.989988,0.950156,0.789757,7.770858,...,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902
6231,2024-12-31 00:00:00+00:00,1.537346,1.851404,1.426810,0.867157,24.537416,7.081013,0.952036,0.792289,7.775136,...,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902
6229,2025-01-01 00:00:00+00:00,1.550713,1.845641,1.381803,0.879927,23.194738,6.919368,0.938662,0.788186,7.767988,...,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000
6228,2025-01-02 00:00:00+00:00,1.608766,1.874190,1.411317,0.876733,24.185942,7.124239,0.946272,0.787828,7.776083,...,0.000000,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951
6232,2025-01-03 00:00:00+00:00,1.560774,1.871500,1.362973,0.883546,24.057713,6.962848,0.939616,0.792183,7.770098,...,0.000000,0.000000,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.43s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,hkd_10,huf_10,jpy_10,krw_10,nok_10,nzd_10,pln_10,sek_10,sgd_10,zar_10
1776,2024-12-12 00:00:00+00:00,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,...,7.7828,391.61,151.67,1396.59,11.0643,1.6991,4.0884,10.9443,1.3437,18.1522
1783,2024-12-13 00:00:00+00:00,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,7.7837,389.84,150.20,1397.57,11.0590,1.6922,4.0674,10.9051,1.3406,18.0626
5469,2024-12-16 00:00:00+00:00,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,7.7808,394.93,150.13,1404.33,11.0964,1.6981,4.0835,10.9708,1.3456,18.1820
2150,2024-12-17 00:00:00+00:00,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,7.7833,394.45,149.64,1403.06,11.0626,1.7002,4.0880,11.0117,1.3451,18.1035
1452,2024-12-18 00:00:00+00:00,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,7.7840,394.96,151.15,1417.54,11.0756,1.7144,4.0858,11.0065,1.3472,18.1738
5183,2024-12-19 00:00:00+00:00,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,7.7824,390.70,150.40,1418.04,11.0626,1.7064,4.0555,10.9198,1.3409,18.0671
5730,2024-12-20 00:00:00+00:00,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,7.7794,391.60,150.51,1419.74,11.0996,1.7118,4.0327,10.8903,1.3396,18.0584
754,2024-12-23 00:00:00+00:00,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,7.7748,389.15,150.83,1432.14,11.1123,1.7061,4.0304,10.9160,1.3396,17.8194
641,2024-12-24 00:00:00+00:00,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,7.7760,390.57,151.74,1431.85,11.1442,1.7178,4.0541,10.9561,1.3411,17.8782
1202,2024-12-27 00:00:00+00:00,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866
